# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [8]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline


from newsapi import NewsApiClient


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/keanumanson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [10]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [11]:
# Fetch the Bitcoin news articles
btc_articles = newsapi.get_everything(
    q="Bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# total articles
print(f"Total articles about Bitcoin: {btc_articles['totalResults']}")

# sample article
btc_articles["articles"][0]

Total articles about Bitcoin: 9236


{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': 'Justine Calma',
 'title': 'Why fossil fuel companies see green in Bitcoin mining projects',
 'description': 'ExxonMobil and other fossil fuel companies have turned to Bitcoin mining to address a gas problem. But their plans come with risks.',
 'url': 'https://www.theverge.com/2022/5/4/23055761/exxonmobil-cryptomining-bitcoin-methane-gas',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/BZdljrBbt8tBl6oCCnckcDKqe6g=/0x90:4608x2503/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/23435975/1240300988.jpg',
 'publishedAt': '2022-05-04T12:00:00Z',
 'content': 'A Bitcoin mining site powered by otherwise lost gas from an oil well near Linden, Texas, on April 4th, 2022. | Photo by FRANCOIS PICARD/AFP via Getty Images\r\n\n \n\n Of all the corporate climate hype fl… [+6544 chars]'}

In [12]:
# Fetch the Ethereum news articles
eth_articles = newsapi.get_everything(
    q="Ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# total articles
print(f"Total articles about Ethereum: {eth_articles['totalResults']}")

# sample article
eth_articles["articles"][0]

Total articles about Ethereum: 4966


{'source': {'id': 'the-verge', 'name': 'The Verge'},
 'author': 'Emma Roth',
 'title': 'US blames North Korean hacker group for $625 million Axie Infinity theft',
 'description': 'The US Department of Treasury has attributed the $625 Axie Infinity theft to Lazarus, a North Korean hacking group with ties to the country’s intelligence agency.',
 'url': 'https://www.theverge.com/2022/4/14/23025739/north-korean-hacker-lazarus-axie-infinity-cryptocurrency-hack-theft-us-blames',
 'urlToImage': 'https://cdn.vox-cdn.com/thumbor/n8TGCqcud7UHPIbtgxLi10Owa5I=/0x52:1018x585/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/23374959/Screen_Shot_2022_03_30_at_10.10.00_AM.png',
 'publishedAt': '2022-04-14T21:43:57Z',
 'content': 'The US Department of Treasury says Lazarus is behind the attack\r\nThe US Treasury Department blames North Korean hacking group Lazarus for stealing $625 million in cryptocurrency from the Ronin networ… [+1853 chars]'}

In [15]:
# Create the Bitcoin sentiment scores DataFrame

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialized the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

btc_sentiments = []

for article in btc_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10] #first top 10??
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame for sentiments
btc_df = pd.DataFrame(btc_sentiments)

# ReorderingDataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/keanumanson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,date,text,compound,positive,negative,neutral
0,2022-05-04,A Bitcoin mining site powered by otherwise los...,-0.0516,0.056,0.061,0.882
1,2022-04-08,Its set to open later this year\r\nIf you buy ...,-0.4404,0.000,0.081,0.919
2,2022-04-14,Wikipedia editors have voted in favor of dropp...,0.4019,0.088,0.000,0.912
3,2022-05-02,Warren Buffett has always been a bitcoin skept...,-0.3269,0.085,0.143,0.772
4,2022-04-05,"You might have heard of crypto Twitter, the co...",-0.2732,0.000,0.057,0.943


In [16]:
# Create the Ethereum sentiment scores DataFrame
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialized the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

eth_sentiments = []

for article in eth_articles["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10] #first top 10??
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame for sentiments
eth_df = pd.DataFrame(eth_sentiments)

# ReorderingDataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/keanumanson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,date,text,compound,positive,negative,neutral
0,2022-04-14,The US Department of Treasury says Lazarus is ...,-0.7845,0.088,0.231,0.681
1,2022-04-13,"18 with 15 posters participating, including st...",0.1027,0.084,0.071,0.844
2,2022-04-15,"Opera launched its dedicated ""Crypto Browser"" ...",0.7351,0.188,0.000,0.812
3,2022-05-02,A multi-billion dollar cryptocurrency company ...,-0.2263,0.046,0.075,0.879
4,2022-05-04,When Bored Ape Yacht Club creators Yuga Labs a...,-0.2732,0.000,0.055,0.945


In [22]:
# Describe the Bitcoin Sentiment
btc_sentiment_stats = btc_df.describe()
btc_sentiment_stats

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.146545,0.062400,0.030840,0.906780
std,0.363230,0.066934,0.049395,0.083026
min,-0.804900,0.000000,0.000000,0.693000
25%,0.000000,0.000000,0.000000,0.856000
50%,0.000000,0.057000,0.000000,0.918000
75%,0.411525,0.093750,0.063500,1.000000
max,0.865500,0.263000,0.222000,1.000000


In [23]:
# Describe the Ethereum Sentiment
eth_sentiment_stats = eth_df.describe()
eth_sentiment_stats

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.201329,0.088900,0.037000,0.874100
std,0.407521,0.090292,0.048876,0.097539
min,-0.784500,0.000000,0.000000,0.594000
25%,-0.115300,0.000000,0.000000,0.812750
50%,0.232250,0.073500,0.000000,0.897500
75%,0.514625,0.135000,0.063750,0.937250
max,0.952000,0.378000,0.233000,1.000000


In [41]:
def coin_sentiment_results():
    print('Mean Positive score:')
    print('Btc', btc_sentiment_stats['positive']['mean'])
    print('Eth', eth_sentiment_stats['positive']['mean'])

    print('\nHighest compound score:') #highest... not mean.
    print('Btc', btc_sentiment_stats['compound']['mean'])
    print('Eth', eth_sentiment_stats['compound']['mean'])

    print('\nHighest positive score:') #max? is that what we want
    print('Btc', btc_sentiment_stats['positive']['max'])
    print('Eth', eth_sentiment_stats['positive']['max'])

coin_sentiment_results()

Mean Positive score:
Btc 0.06239999999999999
Eth 0.0889

Highest compound score:
Btc 0.14654499999999998
Eth 0.20132899999999995

Highest positive score:
Btc 0.263
Eth 0.378


### Questions:

Q: Which coin had the highest mean positive score?

A: Eth has the highest mean positive score

Q: Which coin had the highest compound score?

A: Eth ?

Q. Which coin had the highest positive score?

A: Eth?

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [19]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [11]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer() 

# Create a list of stopwords
stop_words = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    result = [lemmatizer.lemmatize(word) for word in words]
    
    
    # Remove the stop words
    output = [word.lower() for word in words if word.lower() not in sw.union(sw_addons)]

    
    return tokens


#example
#with custom stop words?
def clean_text(article):
    
    # Define a set of stopwords using `stopwords.words()`
    sw = set(stopwords.words('english'))
    
    # Create custom stopwords
    sw_addons = {'said', 'sent', 'found', 'including', 'today', 'announced', 'week', 'basically', 'also'}

    # Define the regex parameters
    regex = re.compile("[^a-zA-Z ]")

    # Apply regex parameters to article
    re_clean = regex.sub('', article)

    # Apply `word_tokenize` to the regex scrubbed text
    words = word_tokenize(re_clean)

    # Create list of lower-case words that are not in the stopword set
    output = [word.lower() for word in words if word.lower() not in sw.union(sw_addons)]
    
    # Return the final list
    return output

#plain 
def clean_text(article):
    
    # Define a set of stopwords using `stopwords.words()`
    sw = set(stopwords.words('english'))

    # Define the regex parameters
    regex = re.compile("[^a-zA-Z ]")

    # Apply regex parameters to article
    re_clean = regex.sub('', article)

    # Apply `word_tokenize` to the regex scrubbed text
    words = word_tokenize(re_clean)

    # Create list of lower-case words that are not in the stopword set
    output = [word.lower() for word in words if word.lower() not in sw]
    
    # Return the final list
    return output


In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---